In [1]:
from sqlalchemy import create_engine, String, Float, text, Binary, bindparam
from sqlalchemy.orm import Session, relationship, backref
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import MetaData, Column, String, Integer, LargeBinary, PrimaryKeyConstraint, ForeignKey, DateTime, Table, Boolean
from sqlalchemy.ext.declarative import declarative_base
from typing import Dict
from sqlalchemy.orm import sessionmaker
from datetime import datetime
import pytz
Base = declarative_base()


In [1]:
# First, we are going to import SQLAlchemy and create a connection to the database.
# This is running inside of Docker, in which we have already installed the Splice ODBC Driver.
from sqlalchemy import create_engine as create_connection

# Database connection info
db_user = 'mlmanager'
db_password = 'admin'
db_host = 'localhost' # special DNS allows us to access ports on the host (specific to OS X dockerd)
# on linux dockerd it will just be localhost, as the network should be configured to be --net host
db_port = 1527
db_database = 'splicedb'

# create SQLAlchemy connection with our SQL dialect. tell it to 
engine = create_connection('splicemachinesa://{user}:{pw}@{host}:{port}/{db}'.format(user=db_user,
                                                                                     pw=db_password, 
                                                                                     host=db_host, 
                                                                                     port=db_port,
                                                                                     db=db_database))

[['DRIVER={SpliceODBCDriver};Database=splicedb;URL=localhost;PORT=1527;UID=mlmanager;PWD=***'], {}]


In [7]:
from sqlalchemy_views import CreateView
from sqlalchemy import Column, String, Integer, LargeBinary, PrimaryKeyConstraint, ForeignKey, DateTime, Boolean, Table
from sqlalchemy.sql import text
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

live_model_status = Table('live_model_status', Base.metadata)
definition = text("""
select mm.RUN_UUID, mm.action,
CASE when ((sta.tableid is null or st.triggerid is NULL or (mm.TRIGGERID_2 is not NULL and st2.triggerid is NULL)) and mm.ACTION='DEPLOYED')
then 'Table or Trigger Missing' else mm.ACTION
end as deployment_status,
mm.TABLEID, mm.TRIGGER_TYPE, mm.TRIGGERID, mm.TRIGGERID_2, mm.DB_ENV, mm.db_user, mm.action_date

from mlmanager.model_metadata mm
left outer join sys.systables sta using (tableid)
left outer join sys.systriggers st on (mm.triggerid=st.triggerid)
left outer join sys.systriggers st2 on (mm.triggerid_2=st2.triggerid)
""")
live_model_status_view = CreateView(live_model_status, definition)

with engine.begin() as cnx:
    cnx.execute(live_model_status_view)
    

In [8]:
%%time
from sqlalchemy import inspect as peer_into_splice_db
inspector = peer_into_splice_db(engine)
inspector.get_view_names()

CPU times: user 1.31 ms, sys: 1.2 ms, total: 2.51 ms
Wall time: 8.69 ms


['live_model_status']

In [3]:
engine.execute('create table foo(a int)')
dir(engine)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_connection_cls',
 '_contextual_connect',
 '_echo',
 '_execute_clauseelement',
 '_execute_compiled',
 '_execute_default',
 '_execution_options',
 '_has_events',
 '_optional_conn_ctx_manager',
 '_run_visitor',
 '_should_log_debug',
 '_should_log_info',
 '_trans_ctx',
 '_wrap_pool_connect',
 'begin',
 'connect',
 'contextual_connect',
 'create',
 'dialect',
 'dispatch',
 'dispose',
 'driver',
 'drop',
 'echo',
 'engine',
 'execute',
 'execution_options',
 'get_execution_options',
 'has_table',
 'hide_parameters',
 'logger',
 'logging_name',
 'name',
 'pool',
 'raw_connection',
 'run_callable',
 'scalar',
 'schema_for_objec

In [4]:
engine.commit()

AttributeError: 'Engine' object has no attribute 'commit'

In [10]:
class SysTables(Base):
    """
    System Table for managing tables
    """
    __tablename__: str = "systables"
    __table_args__: dict = {"schema": "sys"}
    TABLEID: Column = Column(String(1), nullable=False, primary_key=True)
    TABLENAME: Column = Column(String(100), nullable=False)
    TABLETYPE: Column = Column(String(1), nullable=False)
    SCHEMAID: Column = Column(String(1), nullable=False)
    LOCKGRANULARITY: Column = Column(String(100), nullable=False)
    VERSION: Column = Column(String(100), nullable=True)
    COLSEQUENCE: Column = Column(String(100), nullable=False)
    DELIMITED: Column = Column(Integer, nullable=True)
    ESCAPED: Column = Column(String(100), nullable=True)
    LINES: Column = Column(String(100), nullable=True)
    STORED: Column = Column(String(100), nullable=True)
    LOCATION: Column = Column(String(100), nullable=True)
    COMPRESSION: Column = Column(String(100), nullable=True)
    IS_PINNED: Column = Column(Boolean, nullable=False)
    PURGE_DELETED_ROWS: Column = Column(Boolean, nullable=False)


In [11]:
class SysUsers(Base):
    
    """
    System Table for managing users
    """
    __tablename__: str = "sysusers"
    __table_args__: Dict[str,str] = {"schema": "sys"}
    USERNAME: Column = Column(String(100), nullable=False)
    HASHINGSCHEME: Column = Column(String(5000), nullable=False, primary_key=True)
    PASSWORD: Column = Column(String(5000), nullable=False)
    LASTMODIFIED: Column = Column(DateTime, nullable=False)
        

In [12]:
class SysTriggers(Base):
    """
    System Table for managing triggers
    """
    __tablename__: str = "systriggers"
    __table_args__: Dict[str,str] = {"schema": "sys"}
    TRIGGERID: Column = Column(String(1), primary_key=True, nullable=True)
    TRIGGERNAME: Column = Column(String(1000), nullable=True)
    SCHEMAID: Column = Column(String(1), nullable=True)
    CREATIONTIMESTAMP: Column = Column(DateTime, nullable=True)
    EVENT: Column = Column(String(1), nullable=True)
    FIRINGTIME: Column = Column(String(1), nullable=True)
    TYPE: Column = Column(String(1), nullable=True)
    STATE: Column = Column(String(1), nullable=True)
    TABLEID: Column = Column(String(1), nullable=True)
    WHENSTMTID: Column = Column(String(1), nullable=False)
    ACTIONSTMTID: Column = Column(String(1), nullable=False)
    REFERENCEDCOLUMNS: Column = Column(String(5000), nullable=False) 
    TRIGGERDEFINITION: Column = Column(String(5000), nullable=False) 
    REFERENCINGOLD: Column = Column(Boolean, nullable=False)
    REFERENCINGNEW: Column = Column(Boolean, nullable=False)
    OLDREFERENCINGNAME: Column = Column(String(100), nullable=False)
    NEWREFERENCINGNAME: Column = Column(String(100), nullable=False)
    WHENCLAUSETEXT: Column = Column(String(5000), nullable=False)

In [22]:
class ModelMetadata(Base):
    """
    Table for storing metadata information about the deployed models.
    """
    __tablename__: str = "model_metadata"
    __table_args__: Dict[str,str] = {"schema": "mlmanager"}
    run_uuid: Column = Column(String(32), primary_key=True)
    status: Column = Column(String(50), nullable=False)
    deployed_to: Column = Column(String(250), ForeignKey(SysTables.TABLEID), nullable=False) #FIXME: foreign key sys.systables
    trigger_id: Column = Column(String(250), ForeignKey(SysTriggers.TRIGGERID), nullable=False) #FIXME: foreign key sys.systriggers
    trigger_id_2: Column = Column(String(250), ForeignKey(SysTriggers.TRIGGERID), nullable=True) # Some models have 2 triggers #FIXME: foreign key sys.systriggers
    db_env: Column = Column(String(100), nullable=True) # Dev, QA, Prod etc
    deployed_by: Column = Column(String(250), ForeignKey(SysUsers.USERNAME), nullable=False) #FIXME: foreign key sys.sysusers
    deployed_date: Column = Column(DateTime, default=datetime.now(tz=pytz.utc), nullable=False)

    deploy_endpoint: relationship = relationship(SysTables, backref=backref('model_metadata'))
    trigger_1: relationship = relationship(SysTriggers, backref=backref('model_metadata'))
    deploy_user: relationship = relationship(SysUsers, backref=backref('model_metadata'))

In [23]:
tables = [SysTables.__table__, SysUsers.__table__,SysTriggers.__table__, ModelMetadata.__table__]
Base.metadata.create_all(engine, tables=tables, 
                         checkfirst=True)

ProgrammingError: (pyodbc.ProgrammingError) ('42Y08', '[42Y08] [Splice Machine][Splice] (370) Foreign key constraints are not allowed on system tables. (370) (SQLExecDirectW)')
[SQL: 
CREATE TABLE mlmanager.model_metadata (
	run_uuid VARCHAR(32) NOT NULL, 
	status VARCHAR(50) NOT NULL, 
	deployed_to VARCHAR(250) NOT NULL, 
	trigger_id VARCHAR(250) NOT NULL, 
	trigger_id_2 VARCHAR(250), 
	db_env VARCHAR(100), 
	deployed_by VARCHAR(250) NOT NULL, 
	deployed_date TIMESTAMP NOT NULL, 
	PRIMARY KEY (run_uuid), 
	FOREIGN KEY(deployed_to) REFERENCES sys.systables ("TABLEID"), 
	FOREIGN KEY(trigger_id) REFERENCES sys.systriggers ("TRIGGERID"), 
	FOREIGN KEY(trigger_id_2) REFERENCES sys.systriggers ("TRIGGERID"), 
	FOREIGN KEY(deployed_by) REFERENCES sys.sysusers ("USERNAME")
)

]
(Background on this error at: http://sqlalche.me/e/f405)

In [44]:

m = MetaData(schema='SPLICE')
a_base = automap_base(metadata=m)
a_base.prepare(engine, reflect=True) # FAILURE
# Triggers = a_base.classes.systriggers 
# Users = a_base.classes.sysusers
# Tables = a_base.classes.systables
a_base.classes

NotImplementedError: 

In [11]:
SessionMaker: sessionmaker = sessionmaker(bind=engine)


# create a Session
session = SessionMaker()
session.query(SysTables).all()

In [ ]:
SessionMaker.